<a href="https://colab.research.google.com/github/marhooo/AKMMH/blob/main/web_%D0%A4%D0%91_52%D0%BC%D0%BF_%D0%9F%D0%B5%D1%88%D0%BA%D0%BE%D0%B2%D0%B0_%D0%9C%D0%B0%D1%80%D0%B3%D0%B0%D1%80%D0%B8%D1%82%D0%B0_%D0%9B%D0%B0%D0%B1_1_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Імпорт бібліотек
Для роботи з ОС та геолокаціями необхідні наступні бібліотеки:

In [7]:
!pip install user-agents geoip2
!wget https://github.com/P3TERX/GeoLite.mmdb/raw/download/GeoLite2-Country.mmdb

import pandas as pd
import re
from datetime import datetime
from user_agents import parse
import geoip2.database

--2026-02-16 20:32:07--  https://github.com/P3TERX/GeoLite.mmdb/raw/download/GeoLite2-Country.mmdb
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/P3TERX/GeoLite.mmdb/download/GeoLite2-Country.mmdb [following]
--2026-02-16 20:32:17--  https://raw.githubusercontent.com/P3TERX/GeoLite.mmdb/download/GeoLite2-Country.mmdb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9672173 (9.2M) [application/octet-stream]
Saving to: ‘GeoLite2-Country.mmdb.2’

GeoLite2-Country.mm 100%[===================>]   9.22M  --.-KB/s    in 0.07s   

2026-02-16 20:32:17 (129 MB/s) - ‘GeoLite2-Country.mmdb.2’ saved [9672173

## Парсинг файлу

Це необхідно для зчитування логів з файлу та розбиття їх на поля (IP, час, запит тощо) в таблияний вигляд. Оскільки всі логи в файлі access.log мають однакову структуру: ІР-адреса -> час-> запит ->статус-код тощо, то це можна виконати з використанням регулярних виразів.

In [9]:
def parse_line(line):
    regex = r'(?P<IP>\d+\.\d+\.\d+\.\d+) - - \[(?P<Time>.*?)\] "(?P<Request>.*?)" (?P<Status>\d+) (?P<Size>\d+) "(?P<Referer>.*?)" "(?P<User_Agent>.*?)"'
    match = re.search(regex, line)
    if match:
        data = match.groupdict()
        # Конвертація часу у формат datetime для зручності аналізу
        dt = datetime.strptime(data['Time'], '%d/%b/%Y:%H:%M:%S %z')
        data['Date'] = dt.date()
        data['Hour'] = dt.hour
        return data
    return None

with open('access.log', 'r') as f:
    log_entries = [parse_line(line) for line in f if parse_line(line)]

df = pd.DataFrame(log_entries)
df['Status'] = df['Status'].astype(int)
df['Size'] = df['Size'].astype(int)

Оскільки логи розпаршені, тепер можна працювати безпосередню з цими даними.


In [12]:
#а. Визначити кількість користувачів за днями
daily_users = df.groupby('Date')['IP'].nunique().reset_index(name='Unique_Users')
print("Кількість унікальних користувачів за днями:")
print(daily_users)

Кількість унікальних користувачів за днями:
          Date  Unique_Users
0   2012-06-20            21
1   2012-06-21            69
2   2012-06-22            68
3   2012-06-23            83
4   2012-06-24            78
5   2012-06-25            73
6   2012-06-26            90
7   2012-06-27            73
8   2012-06-28            88
9   2012-06-29            93
10  2012-06-30            98
11  2012-07-01            82
12  2012-07-02            30


In [13]:
# b. Ранжувати користувачів за User-Agent
ua_ranking = df.groupby('User_Agent')['IP'].nunique().sort_values(ascending=False).reset_index(name='Unique_IP_Count')
print("Ранжування User-Agent (топ-10):")
ua_ranking.head(10)

Ранжування User-Agent (топ-10):


,User_Agent,Unique_IP_Count
0,Mozilla/5.0 (compatible; Baiduspider/2.0; +htt...,193
1,Mozilla/5.0 (compatible; MJ12bot/v1.4.3; http:...,32
2,Mozilla/5.0 (compatible; AhrefsBot/3.1; +http:...,22
3,Mozilla/5.0 (Windows NT 5.1; rv:6.0.2) Gecko/2...,15
4,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,13
5,Baiduspider+(+http://www.baidu.com/search/spid...,12
6,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,10
7,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,7
8,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,7
9,Mozilla/4.0 (compatible; MSIE 6.0; MSIE 5.5; W...,6


In [14]:
#c. Ранжувати користувачів за операційними системами

from user_agents import parse

# Отримання назви ОС з User-Agent
df['OS'] = df['User_Agent'].apply(lambda x: parse(x).os.family)

# Обчислення унікальних IP для кожної ОС
os_ranking = df.groupby('OS')['IP'].nunique().sort_values(ascending=False).reset_index(name='Unique_IP_Count')
print("Ранжування за операційними системами:")
print(os_ranking)

Ранжування за операційними системами:
         OS  Unique_IP_Count
0     Other              327
1   Windows              119
2   Android                5
3       iOS                3
4     Linux                2
5    Ubuntu                2
6   FreeBSD                1
7  Mac OS X                1
8      SUSE                1


In [15]:
# d. Ранжувати користувачів за країною запиту

import geoip2.database

def get_country(ip):
    try:
        with geoip2.database.Reader('GeoLite2-Country.mmdb') as reader:
            return reader.country(ip).country.name
    except:
        return "Unknown"

# Додаємо колонку з країною [cite: 13, 14]
df['Country'] = df['IP'].apply(get_country)

# Ранжуємо країни за кількістю унікальних IP [cite: 15]
country_ranking = df.groupby('Country')['IP'].nunique().sort_values(ascending=False).reset_index(name='Unique_IP_Count')
print("Ранжування за країнами:")
print(country_ranking)

Ранжування за країнами:
           Country  Unique_IP_Count
0            China              193
1    United States              102
2            Japan               22
3           Russia               22
4           France               14
5          Germany               13
6          Ukraine               12
7   United Kingdom                6
8      Netherlands                6
9        Indonesia                5
10          Brazil                5
11         Czechia                5
12          Canada                4
13           India                4
14          Israel                4
15          Norway                4
16        Thailand                2
17           Spain                2
18        Malaysia                2
19         Ecuador                2
20          Sweden                2
21         Belgium                1
22       Argentina                1
23      Luxembourg                1
24       Hong Kong                1
25            Iran                1
26  

In [16]:
#e. Виокремити пошукових ботів

# Список патернів для ідентифікації ботів
bot_patterns = ['bot', 'crawler', 'spider', 'google', 'bing', 'yandex', 'mj12bot']

def is_bot(ua):
    ua_low = ua.lower()
    for pattern in bot_patterns:
        if pattern in ua_low:
            return pattern
    return "Human"

df['Bot_Type'] = df['User_Agent'].apply(is_bot)

bots_found = df[df['Bot_Type'] != "Human"]['Bot_Type'].value_counts().reset_index(name='Requests_Count')
print("Виявлені пошукові боти та інструменти:")
print(bots_found)

Виявлені пошукові боти та інструменти:
  Bot_Type  Requests_Count
0      bot             988
1   spider             591
2  crawler              27
3   google               2


In [18]:
# f. Детектувати аномалії

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

# 1. Ознаки для кожного IP
anomaly_data = df.groupby('IP').agg({
    'Request': 'count',
    'Status': lambda x: (x >= 400).sum(),  # Кількість помилок
    'Size': 'sum'
}).rename(columns={'Request': 'Total_Requests', 'Status': 'Error_Count'})

# 2. Стандартизація
scaler = StandardScaler()
scaled_data = scaler.fit_transform(anomaly_data)

# 3. Навчання моделі
model = IsolationForest(contamination=0.02, random_state=42) # припускаємо 2% аномалій
anomaly_data['is_anomaly'] = model.fit_predict(scaled_data)

print("Виявлені аномальні IP-адреси (підозріла активність):")
print(anomaly_data[anomaly_data['is_anomaly'] == -1])

Виявлені аномальні IP-адреси (підозріла активність):
                 Total_Requests  Error_Count     Size  is_anomaly
IP                                                               
1.202.218.8                 130          130    63700          -1
108.21.95.73                  6            0  1510291          -1
176.31.247.216               62           54    30337          -1
178.154.210.252              72           22   124115          -1
208.115.113.91               60           29  1488102          -1
66.249.72.65                 56           27   297222          -1
89.123.3.221                 58           19  2235380          -1
95.108.150.235               72           24    44594          -1
95.108.151.244              132           12    87086          -1
